https://www.dropbox.com/s/k585ptm20y1cl86/ETH_Ballbot_Thesis.pdf?dl=0

In [1]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, LagrangesMethod, Lagrangian
from sympy.physics import mechanics
import sympy as sp
init_vprinting()

import numpy as np
import math
from time import time, sleep

In [40]:
def hat(v):
    a, b, c = v
    return Matrix([[0, -c, b], [c, 0, -a], [-b, a, 0]])

def Ad(H):
    R = H[:3, :3]
    o = H[:3, 3]
    Z = zeros(3)
    ans = [[R, Z], [hat(o) @ R, R]]
    return Matrix(ans)

def Hinv(H):
    R = H[:3, :3]
    p = H[:3, 3]
    z = Matrix(np.zeros((1, 3)))
    return Matrix([[R.T, -R.T @ p], [z, Matrix([1])]])

def jacobian(v, dq):
    if type(v) != Matrix:
        v = Matrix([v])
    n = len(v)
    m = len(dq)
    ans = np.zeros((n, m), dtype=object)
    
    zeros_slist = [(var, 0) for var in dq]
    for i in range(n):
        for j in range(m):
            ans[i, j] = v[i].subs(dq[j], 1).subs(zeros_slist)
    return Matrix(ans)

def partial(arg, x):
    lst = []
    for v in x:
        lst.append(arg.diff(v))
    return Matrix([lst])

def Rx(th):
    return Matrix([[1, 0, 0], [0, cos(th), -sin(th)], [0, sin(th), cos(th)]])

def Ry(th):
    return Matrix([[cos(th), 0, sin(th)], [0, 1, 0], [-sin(th), 0, cos(th)]])

def Rz(th):
    return Matrix([[cos(th), -sin(th), 0], [sin(th), cos(th), 0], [0, 0, 1]])

def nice_display(mat):
    return Matrix(np.array(mat, dtype=float).round(5))


In [3]:
t = symbols("t")
q = Matrix(dynamicsymbols("theta_x theta_y theta_z phi_x phi_y"))
dq = q.diff(t)

thx, thy, thz, phix, phiy = q
dthx, dthy, dthz, dphix, dphiy = dq

Tvec = Matrix(symbols("T_1 T_2 T_3"))
T1, T2, T3 = Tvec

if False:
    mk, mw, ma, rk, rw, ra = symbols("m_k m_w m_a r_k r_w r_a")
    l = symbols("l") # Height of COM
    igear = symbols("i_Gear")
    g = symbols("g")
    
    iK, iAx, iAy, iAz, iW = symbols("I_K I_Ax I_Ay I_Az I_W")
else:
    # Parameters
    alpha = rad(45)
    b1, b2, b3 = 0, rad(120), rad(240)

    mk = 2.29
    mw = 3
    ma = 9.2
    rk = 0.125
    rw = 0.06
    ra = 0.1
    l = 0.339
    igear = 26
    
    iK = 0.0239
    iAx = 2.026
    iAy = 2.025
    iAz = 0.092
    iW = 0.00315
    
    g = 9.8
    
IIK = diag(iK, iK, iK)
LIK = IIK
AIA = diag(iAx, iAy, iAz)


In [4]:
RIL = Rz(thz)
RLI = RIL.T

RLA = Ry(thy) * Rx(thx)
RAL = RLA.T

RIA = RIL * RLA
RAI = RIA.T

In [5]:
# Angular velocity of the ball 
LWK = Matrix([dphix, dphiy, 0])
# Angular speed of the body
AWA = Matrix([dthx - sin(thy)*dthz, 
              cos(thx)*dthy + cos(thy)*sin(thx)*dthz, 
              -sin(thx)*dthy + cos(thx)*cos(thy)*dthz])

In [75]:
Ry(alpha) * Matrix([1, 0, 0])

⎡ √2 ⎤
⎢ ── ⎥
⎢ 2  ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢-√2 ⎥
⎢────⎥
⎣ 2  ⎦

In [95]:
# Speed of the omniwheels:

# Motor Axis
MW_gen = Ry(alpha) * Matrix([1, 0, 0])
MW1 = simplify(Rz(b1) * MW_gen)
MW2 = simplify(Rz(b2) * MW_gen)
MW3 = simplify(Rz(b3) * MW_gen)

# Vec from center of ball to pmniwheel contact point
APK_gen = Ry(alpha) * Matrix([0, 0, rk])
# Vec of tangential omniwheel speed
Ad_gen = Matrix([0, 1, 0])

APK1 = Rz(b1) * APK_gen
Ad1 = Rz(b1) * Ad_gen

APK2 = Rz(b2) * APK_gen
Ad2 = Rz(b2) * Ad_gen

APK3 = Rz(b3) * APK_gen
Ad3 = Rz(b3) * Ad_gen

# Angular velocity of ball w.r.t. body in A frame
AwK = RAL * LWK - AWA

# Solving for A_w_Wi
AwW1 = AwK.cross(APK1).dot(Ad1) / rw
AwW2 = AwK.cross(APK2).dot(Ad2) / rw
AwW3 = AwK.cross(APK3).dot(Ad3) / rw

AWW1 = AwW1 + MW1.dot(AWA) / MW1.norm()
AWW2 = AwW2 + MW2.dot(AWA) / MW2.norm()
AWW3 = AwW3 + MW3.dot(AWA) / MW3.norm()

# Checked AWW2. Is correct.

In [130]:
AwK.cross(APK2).dot(Ad2)

In [129]:
AwK.cross(APK3).dot(Ad3)

In [97]:
# Speed vector of the center of the ball
IBP = Matrix([0, 0, rk])
IWK = RIL * LWK
IdrP = IWK.cross(IBP)

In [98]:
# Ball Energy
Tk = mk/2 * IdrP.T*IdrP + LWK.T * LIK * LWK / 2
Tk = Tk[0]

# Body Energy
G = Matrix([0, 0, -g])
ArPSA = Matrix([0, 0, l]) # Not sure (vec from center of ball P to COM of body in A coords)

Ta = ma/2 * IdrP.dot(IdrP) + ma*(RAI*IdrP).dot(AWA.cross(ArPSA)) + AWA.dot(AIA * AWA)/2
Va = -ma*G.T * RIA * ArPSA
Va = Va[0]

# Omniwheel Energy
TWs = iW * simplify(AWW1**2 + AWW2**2 + AWW3**2)


In [99]:
JT1 = jacobian(AwW1, dq)
JT2 = jacobian(AwW2, dq)
JT3 = jacobian(AwW3, dq)

TC1 = MW1/MW1.norm() * -T1
TC2 = MW2/MW2.norm() * -T2
TC3 = MW3/MW3.norm() * -T3

JTC = jacobian(AWA, dq)

In [100]:
f_NP = JT1.T * T1 + JT2.T * T2 + JT3.T * T3 + JTC.T * (TC1 + TC2 + TC3)

In [111]:
nice_display(partial(JT1.T * T1 + JT2.T * T2 + JT3.T * T3, Tvec).subs(xbar_subs))

⎡1.47314   -0.73657  -0.73657⎤
⎢                            ⎥
⎢   0      1.27578   -1.27578⎥
⎢                            ⎥
⎢-1.47314  -1.47314  -1.47314⎥
⎢                            ⎥
⎢-1.47314  0.73657   0.73657 ⎥
⎢                            ⎥
⎣   0      -1.27578  1.27578 ⎦

In [122]:
JT2[4]

In [123]:
JT3[4]

In [101]:
xbar_subs = [(v, 0) for v in q] + [(v, 0) for v in dq]
xvar = Matrix([q, dq])
u = Tvec

In [102]:
t0 = time()
LM = LagrangesMethod(Tk + Ta + TWs - Va, q)
LM.form_lagranges_equations()
print("Took %.2f seconds"%(time() - t0))

Took 3.12 seconds


In [103]:
Mlin_sp = LM.mass_matrix.subs(xbar_subs)
Minv_sp = Mlin_sp.inv()

Glin_sp = -LM.forcing.subs([(v, 0) for v in (list(dq) + list(dq.diff(t)))])
dG_sp = np.array(partial(Glin_sp, xvar).subs(xbar_subs).doit(), dtype=float).round(3)

Abot_sp = Minv_sp @ -dG_sp

In [107]:
dT_ext = np.array(partial(f_NP, Tvec).subs(xbar_subs), dtype=float).round(3)
Bbot_sp = Minv_sp @ dT_ext

In [90]:
Abot_sp

⎡20.4605302850238           0          0  0  0  0  0  0  0  0⎤
⎢                                                            ⎥
⎢        0          20.4742364011719   0  0  0  0  0  0  0  0⎥
⎢                                                            ⎥
⎢        0                  0          0  0  0  0  0  0  0  0⎥
⎢                                                            ⎥
⎢-32.8461170899449          0          0  0  0  0  0  0  0  0⎥
⎢                                                            ⎥
⎣        0          -32.8681200727404  0  0  0  0  0  0  0  0⎦

In [57]:
L = Tk + Ta + TWs - Va

LEQ = L.diff(dq).diff(t) - L.diff(q)

Gpart = LEQ.subs([(v, 0) for v in (list(dq) + list(dq.diff(t)))]).doit()
remainder = (LEQ - Gpart).doit()

Cpart = (remainder.subs([(v, 0) for v in dq.diff(t)])).doit()
Mpart = (remainder - Cpart).doit()

Mmat = partial(Mpart, dq.diff(t))
Mlin = Mmat.subs(xbar_subs)
Minv = Mlin.inv()

In [58]:
Abot = np.array(Minv * -dG, dtype=float).round(5)
Matrix(Abot)

⎡20.46053       0      0  0  0  0  0  0  0  0⎤
⎢                                            ⎥
⎢    0      20.47424   0  0  0  0  0  0  0  0⎥
⎢                                            ⎥
⎢    0          0      0  0  0  0  0  0  0  0⎥
⎢                                            ⎥
⎢-32.84612      0      0  0  0  0  0  0  0  0⎥
⎢                                            ⎥
⎣    0      -32.86812  0  0  0  0  0  0  0  0⎦

In [59]:
nice_display(Minv), nice_display(dT_ext)

⎛⎡0.66943      0         0     -1.07467     0    ⎤  ⎡0.766   -0.383  -0.383⎤⎞
⎜⎢                                               ⎥  ⎢                      ⎥⎟
⎜⎢   0      0.66988      0        0      -1.07539⎥  ⎢  0     0.663   -0.663⎥⎟
⎜⎢                                               ⎥  ⎢                      ⎥⎟
⎜⎢   0         0      5.49932     0         0    ⎥, ⎢-0.766  -0.766  -0.766⎥⎟
⎜⎢                                               ⎥  ⎢                      ⎥⎟
⎜⎢-1.07467     0         0     6.19071      0    ⎥  ⎢-1.473  0.737   0.737 ⎥⎟
⎜⎢                                               ⎥  ⎢                      ⎥⎟
⎝⎣   0      -1.07539     0        0      6.19186 ⎦  ⎣  0     -1.276  1.276 ⎦⎠

In [60]:
dT_ext = np.array(partial(f_NP, Tvec).subs(xbar_subs), dtype=float).round(7)
Bbot = np.array(Minv * dT_ext, dtype=float).round(5)
Matrix(Bbot)

⎡2.09594   -1.04797  -1.04797⎤
⎢                            ⎥
⎢   0      1.81635   -1.81635⎥
⎢                            ⎥
⎢-4.21266  -4.21266  -4.21266⎥
⎢                            ⎥
⎢ -9.943    4.9715    4.9715 ⎥
⎢                            ⎥
⎣   0      -8.61284  8.61284 ⎦